# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


In [ ]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

In [ ]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

In [ ]:
dataset_path = '/content/drive/MyDrive/DMEyF/2025/datos/'
dataset_file = 'competencia_01.csv'

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

In [ ]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01 limit 10

Lo siguiente es querer operar dos variables, como por ejemplo sumarla. Esto es sencillo


In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 limit 10

Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null.

In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10

Esto no siempre es deseable y puede ser fácilmente evitable

In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01 limit 10

In [ ]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);


In [ ]:
%%sql
select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [ ]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6
from competencia_01
order by numero_de_cliente, cliente_antiguedad


Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
from competencia_01
limit 10


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [ ]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']


Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [ ]:
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)





Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [ ]:
%%sql
COPY competencia_01 TO '{dataset_path}competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE);
